In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [27]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
warnings.filterwarnings('ignore',message='profile')
warnings.filterwarnings('ignore',message='_astropy_init')
warnings.filterwarnings('ignore',message='ConfigurationDefaultMissingWarning ')

import os
import sys

In [6]:
from astroquery.vizier import Vizier
import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

#import pyvo as vo

In [4]:
Vizier.ucd  = 'phot.flux.density;em.radio.*'

In [22]:
def create_lsm(ra, dec, radius):
    """
    ra     : float in degrees with the right ascension of the target direction
    dec    : float in degrees with the declination of the target direction
    radius : float in degrees with the radius around the target to search for sources
    """
    
    Vizier.ucd  = 'phot.flux.density;em.radio.*'
    radius = str(radius) + 'd'
    
    if -30 >= dec >= -90:
        cat = 'The SUMSS Catalog, Version 2.1r (Feb-16-2012)'
        catalog_list = Vizier.find_catalogs('VIII/81B/sumss212') 
    else:
        cat = 'The NVSS Catalog'
        catalog_list = Vizier.find_catalogs('VIII/65/nvss') 
        
    result = Vizier.query_region(coord.SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='icrs'), 
                                 radius=radius, catalog=catalog_list.keys())
    
    
    for i in range(len(result)):
        if cat==result[i].meta['description']:
            print(i, result[i].meta['description'])
            chosen_cat = i
            
    MyCat = result[result.keys()[chosen_cat]]
    
    return MyCat

In [145]:
lsm = create_lsm(0, -29, 90)

/usr/local/lib/python2.7/dist-packages/astroquery/vizier/core.py:710: UserWarning: VOTABLE parsing raised exception: None:17641:205: ValueError: invalid literal for int() with base 10: 'null' (in row 31, col 'Stage')
  warnings.warn("VOTABLE parsing raised exception: {0}".format(ex))


(88, u'The NVSS Catalog')


In [146]:
lsm

NVSS,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,S1.4,e_S1.4,l_MajAxis,MajAxis,l_MinAxis,MinAxis,f_resFlux
,"""h:m:s""","""d:m:s""",s,arcs,mJy,mJy,,arcs,,arcs,
str14,str11,str11,float32,float32,float64,float32,str1,float32,str1,float32,str2
000000-340709,00 00 00.09,-34 07 09.5,0.48,6.7,2.7,0.5,<,59.8,<,52.4,
000000-382628,00 00 00.13,-38 26 28.5,0.65,11.5,3.1,0.6,<,117.0,<,51.7,
000000-040242,00 00 00.16,-04 02 42.3,0.03,0.6,106.8,3.2,<,18.5,<,16.4,
000000+321233,00 00 00.18,+32 12 33.3,0.31,3.9,3.7,0.4,<,43.8,<,41.0,
000000+491812,00 00 00.19,+49 18 12.6,0.92,9.0,2.4,0.5,<,93.8,<,58.4,
000000-200448,00 00 00.25,-20 04 48.8,0.25,7.9,7.6,1.4,,71.5,<,38.0,
000000+004100,00 00 00.39,+00 41 00.0,0.03,0.6,149.5,5.3,,45.4,,35.7,S*
000000-181729,00 00 00.40,-18 17 29.3,0.54,7.3,2.3,0.5,<,68.5,<,59.7,


In [19]:
# LSM should be of the following format:
# #name ra_d dec_d i q u v emaj_s emin_s pa_d

MyLSM = []
if CatChosen == 'SUMSS':
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['Sp'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))
        
elif CatChosen == 'NVSS':
    
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['S1.4'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))

In [5]:

RA = raw_input("Enter RA of target hh mm ss ")
DEC = raw_input("Enter Dec of target dd mm ss ")

Enter RA of target hh mm ss 0 0 0
Enter Dec of target dd mm ss -31 0 0 


In [6]:
MyRadius = raw_input('Enter search radius e.g. 16m, 16d, 16s ')

Enter rsearch radius e.g. 16m, 16d, 16s 30d


In [7]:
def InputCoord(arg1,arg2):
    '''This takes a user input of the RA and Dec and then 
    convert it to RaTarget and DecTarget in deg for use in the VO search.
    Input: arg1(RA), arg2(Dec) NB: They are str in this case.
    Output: RaTarget,DecTarget
    '''
    hh = float(arg1.split(' ')[0])
    mm = float(arg1.split(' ')[1])/60.
    ss = float(arg1.split(' ')[2])/3600.

    dec_dd = float(arg2.split(' ')[0])
    dec_mm = float(arg2.split(' ')[1])/60.
    dec_ss = float(arg2.split(' ')[2])/3600.

    if dec_dd <0:
        dec = dec_dd-dec_mm-dec_ss
    else:
        dec = dec_dd+dec_mm+dec_ss

    RaTarget = (hh+mm+ss)*15.
    DecTarget = dec
    
    return RaTarget,DecTarget

In [8]:
Ra,Dec = InputCoord(RA, DEC)

In [9]:
Dec 

-31.0

In [10]:
# The commonly used Radio catalogues on Vizier is given by:
# first_cat(VIII/92/first14)   
# nvss_cat(VIII/65/nvss)
# SUMSS_cat(VIII/81B/sumss212)


if (Dec > -90.) and (Dec < -30.):
    print("Recommend one use SUMSS survey ")
    catalog_list = Vizier.find_catalogs('VIII/81B/sumss212') 

    result = Vizier.query_region(coord.SkyCoord(ra=Ra, dec=Dec,unit=(u.deg, u.deg),frame='icrs')
                                 ,radius=MyRadius,catalog=catalog_list.keys())

elif (Dec > -30.) and (Dec < 90.):
    print("Recommend to use NVSS or FIRST if Dec >-10 ")
    catalog_list = Vizier.find_catalogs('VIII/65/nvss') 

    result = Vizier.query_region(coord.SkyCoord(ra=Ra, dec=Dec,unit=(u.deg, u.deg),frame='icrs')
                                 ,radius=MyRadius,catalog=catalog_list.keys())
else:
    print("Check your input ")


Recommend one use SUMSS survey 


In [11]:
# Printing catalogues that are accessible
for i in range(len(result)):
    print i,'-',result[i].meta['description']

0 - Observed radio source identifications
1 - Optical and infrared data
2 - Discoverers, Designations, and Positions of True and Possible Planetary Nebulae (1)
3 - IRAS Point Source Catalogue fluxes (1)
4 - Objects rejected as PN (table 3)
5 - X-ray and radio parameters of XMM-Newton source cross-matched radio sources from NVSS, MGPS2 and SUMSS
6 - Observed data 510 extragalactic radio sources
7 - Derived values
8 - Redshift-corrected data
9 - Redshift-corrected derived values
10 - The RC2 catalogue
11 - General list of objects
12 - Observed and Derived parameters (tables 1, 2 and 4 of the publication)
13 - Main catalogue
14 - Sources close to coverage gap
15 - 2QZ catalogue: includes North and South galactic poles, and 6QZ with another 1657 lines.
16 - Additional spectra when more than 2 spectra were taken
17 - dat.The sample of high-redshift galaxies: spectral indeces, redshifts, magnitudes, sizes and flux density for 2442 galaxies.
18 - Quasars (M_v_<-22.25), BL Lac and AGNs (M_v_>-

In [12]:
indx = int(raw_input("Enter catalogue number to use "))

Enter catalogue number to use 69


In [13]:
MyCat = result[result.keys()[indx]]
print "Catalogue chosen is: ", result[indx].meta['description']
MyCat[:5]

Catalogue chosen is:  The SUMSS Catalog, Version 2.1r (Feb-16-2012)


RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Sp,e_Sp,St,e_St,MajAxis,MinAxis,PA,Mosaic,Nm
"""h:m:s""",arcs,"""d:m:s""",arcs,mJy,mJy,mJy,mJy,arcs,arcs,deg,,
str11,float32,str12,float32,float32,float32,float64,float32,float32,float32,float32,str8,uint8
00 00 00.00,3.6,-31 09 53.10,4.7,13.1,1.4,13.2,1.4,84.9,58.4,162.3,J0000M32,1
00 00 00.03,3.1,-50 28 08.50,3.6,7.4,0.8,7.4,0.8,57.1,46.6,163.8,J0000M52,1
00 00 00.10,2.1,-45 57 21.50,2.5,16.7,1.0,20.9,1.3,70.0,50.1,17.1,J0000M48,2
00 00 00.26,1.8,-47 37 57.70,2.0,28.1,1.3,28.1,1.3,60.6,48.7,158.9,J0000M48,1
00 00 01.72,3.3,-59 36 16.30,3.5,11.1,1.2,11.1,1.2,52.0,47.0,151.8,J0000M60,1


In [17]:
len(MyCat)

50

In [18]:
if "SUMSS" in result[indx].meta['description']:
    CatChosen = 'SUMSS'
elif "NVSS" in result[indx].meta['description']:
    CatChosen = 'NVSS'
else:
    print("Non standard cat chosen may break code")

In [19]:
# LSM should be of the following format:
# #name ra_d dec_d i q u v emaj_s emin_s pa_d

MyLSM = []
if CatChosen == 'SUMSS':
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['Sp'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))
        
elif CatChosen == 'NVSS':
    
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['S1.4'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))

In [20]:
MyLSMHeader = '#format:name ra_d dec_d i q u v emaj_s emin_s pa_d'

In [21]:
np.savetxt('test_LSM.lsm', np.array(MyLSM),header=MyLSMHeader,comments='',
           fmt= '%i %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e')

In [23]:
!ls -lah

total 45G
drwxr-xr-x  7 2020 1000 4.0K Nov 12 11:32 .
drwxr-xr-x 18 2020 1000 4.0K Nov  2 09:34 ..
drwxr-xr-x  8 2020 1000  257 Nov 12 11:20 .git
-rw-r--r--  1 2020 1000  163 Nov 12 11:19 .gitignore
drwxr-xr-x  2 root root  172 Nov 12 11:26 .ipynb_checkpoints
-rw-r--r--  1 2020 1000 1.6K Nov 11 15:03 README.md
-rw-r--r--  1 2020 1000 8.2K Nov 12 11:16 RFIsim.py
-rw-r--r--  1 2020 1000  20K Nov 12 11:30 VO_example_for_Sky_model.ipynb
-rw-r--r--  1 2020 1000  49K Nov 12 11:32 VO_example_for_Sky_model_w_NVSS_n_SUMSS.ipynb
-rw-r--r--  1 2020 root 3.4M Nov 11 18:25 montblanc.log
drwxr-xr-x  3 2020 1000  333 Nov 12 11:20 notebooks
drwxr-xr-x  2 2020 1000   10 Nov  8 16:01 output_data
-rw-r--r--  1 root root  45G Nov 11 18:25 ra=0.0_dec=0.0_int_secs=8_track-time=0.1hrs_nants=64_nchan=4096.h5
-rw-r--r--  1 root root 5.6K Nov 12 11:32 test_LSM.lsm
drwxr-xr-x  6 2020 1000  197 Nov 12 11:16 utils


In [19]:
# Copy the sky model to the path you want

In [281]:
import Tigger

In [313]:
lsm = Tigger.load('test_LSM.lsm')

Loading test_LSM.lsm: ASCII table


In [314]:
lsm

In [315]:
lsm.sources

In [316]:
lsm.sources[0].shape

In [317]:
lsm.sources[0].shape.pa

AttributeError: 'NoneType' object has no attribute 'pa'

In [291]:
np.deg2rad(1.62222e-02)

0.00028313080191702414

In [318]:
!cat test_LSM.lsm

# #format:name ra_d dec_d i q u v emaj_s emin_s pa_d
0 2.56250e-02 -2.99196e+01 1.35000e+01 0.00000e+00 0.00000e+00 0.00000e+00 2.58889e-02 1.42500e-02 1.69100e+02
1 9.83333e-02 -2.99916e+01 1.85000e+01 0.00000e+00 0.00000e+00 0.00000e+00 3.09444e-02 2.05278e-02 1.43400e+02


In [287]:
!cat test_LSM.lsm

#format:name ra_d dec_d i q u v emaj_s emin_s pa_d
0 0.00000e+00 -3.11647e+01 1.31000e-02 0.00000e+00 0.00000e+00 0.00000e+00 2.35833e-02 1.62222e-02 1.62300e+02
1 1.25000e-04 -5.04690e+01 7.40000e-03 0.00000e+00 0.00000e+00 0.00000e+00 1.58611e-02 1.29444e-02 1.63800e+02
2 4.16667e-04 -4.59560e+01 1.67000e-02 0.00000e+00 0.00000e+00 0.00000e+00 1.94444e-02 1.39167e-02 1.71000e+01
3 1.08333e-03 -4.76327e+01 2.81000e-02 0.00000e+00 0.00000e+00 0.00000e+00 1.68333e-02 1.35278e-02 1.58900e+02
4 7.16667e-03 -5.96045e+01 1.11000e-02 0.00000e+00 0.00000e+00 0.00000e+00 1.44444e-02 1.30556e-02 1.51800e+02
5 7.20833e-03 -4.20757e+01 2.95000e-02 0.00000e+00 0.00000e+00 0.00000e+00 2.15556e-02 1.27500e-02 6.30000e+00
6 9.00000e-03 -5.13366e+01 2.12000e-02 0.00000e+00 0.00000e+00 0.00000e+00 1.58611e-02 1.45556e-02 1.22400e+02
7 9.04167e-03 -4.75748e+01 1.25000e-02 0.00000e+00 0.00000e+00 0.00000e+00 1.68333e-02 1.45833e-02 3.97000e+01
8 9.37500e-03 -3.71333e+01 1.32000e-02 0.00000e+00 0.00000e+0

## SUMSS Catalogue 

www.astrop.physics.usyd.edu.au/sumsscat/sumsscat.Mar-11-2008.gz

In [29]:
import pandas as pd

In [237]:
sumss = np.loadtxt('utils/catalogues/sumsscat.Mar-11-2008', dtype=str)

In [238]:
sumss_cols = ['RAJ2000hh', 'RAJ2000mm', 'RAJ2000ss', 'DEJ2000dd', 'DEJ2000mm', 'DEJ2000ss', 
              'e_RAJ2000', 'e_DEJ2000', 'Sp', 'e_Sp', 'St', 'e_St', 'MajAxis', 'MinAxis', 
              'PA', 'x1', 'x2', 'x3', 'Mosaic', 'Nm', 'x4', 'x5']

In [248]:
df = pd.DataFrame(sumss, columns=sumss_cols)

In [249]:
df = df.drop(columns=['e_RAJ2000', 'e_DEJ2000', 'Sp', 'e_Sp', 'e_St', 
                 'x1', 'x2', 'x3', 'Mosaic', 'Nm', 'x4', 'x5'])

df = df.apply(pd.to_numeric, errors='coerce')

In [250]:
df.describe()

,RAJ2000hh,RAJ2000mm,RAJ2000ss,DEJ2000dd,DEJ2000mm,DEJ2000ss,St,MajAxis,MinAxis,PA
count,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000
mean,11.221223,29.441157,29.916419,-52.746891,29.413592,29.905832,62.705213,65.033987,50.078154,90.762212
std,7.627004,17.308951,17.359206,15.225615,17.385063,17.336541,879.781546,17.924319,7.170397,54.965468
min,0.000000,0.000000,0.000000,-89.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000
25%,4.000000,14.000000,14.870000,-65.000000,14.000000,14.900000,12.800000,53.600000,45.500000,41.700000
50%,11.000000,29.000000,29.940000,-51.000000,29.000000,29.900000,21.200000,60.800000,47.600000,90.800000
75%,19.000000,44.000000,44.980000,-39.000000,44.000000,44.900000,45.800000,76.500000,52.400000,139.900000
max,23.000000,59.000000,59.900000,-25.000000,59.000000,59.900000,342000.000000,1156.600000,839.000000,180.000000


In [251]:
df['RA'] = (df['RAJ2000hh'] + df['RAJ2000mm']/60 + df['RAJ2000ss']/3600)*15

In [252]:
df['DEC'] = -1*(np.abs(df['DEJ2000dd']) + df['DEJ2000mm']/60 + df['DEJ2000ss']/3600)

In [253]:
df['Flux'] = df['St']

In [254]:
df = df.drop(columns=['RAJ2000hh', 'RAJ2000mm', 'RAJ2000ss', 
                      'DEJ2000dd', 'DEJ2000mm', 'DEJ2000ss', 'St'])

In [255]:
df.head()

,MajAxis,MinAxis,PA,RA,DEC,Flux
0,84.9,58.4,162.3,0.000000,-31.164750,13.2
1,57.1,46.6,163.8,0.000125,-50.469028,7.4
2,70.0,50.1,17.1,0.000417,-45.955972,20.9
3,60.6,48.7,158.9,0.001083,-47.632694,28.1
4,52.0,47.0,151.8,0.007167,-59.604528,11.1


In [256]:
df.to_csv('utils/catalogues/SUMSS_Catalogue.csv', index=False)

## NVSS Catalogue

ftp://nvss.cv.nrao.edu/pub/nvss/CATALOG/NVSSCatalog.text.gz

In [116]:
fp = open('utils/catalogues/NVSSCatalog.text', 'r')

In [117]:
lines = fp.readlines()

In [135]:
data = []
for line in lines[14:]:
    if 'NVSS' in line or 'RA' in line or 'mJy' in line:
        continue
    else:
        data.append(line.rstrip('\n'))

In [137]:
len(data), len(data)/2

(3546969, 1773484)

In [138]:
data = [data[2*i]+data[2*i+1] for i in range(len(data)/2)]

In [139]:
data = np.array([data[i].split()[:13] for i in range(len(data))])

In [140]:
data.shape

(1773484, 13)

In [141]:
nvss_cols = ['RAJ2000hh', 'RAJ2000mm', 'RAJ2000ss', 'DEJ2000dd', 'DEJ2000mm', 'DEJ2000ss', 
              'Flux', 'MajAxis', 'MinAxis', 'PA', 'Res', 'P_Flux', 'Field']

In [178]:
df = pd.DataFrame(data, columns=nvss_cols)

In [179]:
df.head()

,RAJ2000hh,RAJ2000mm,RAJ2000ss,DEJ2000dd,DEJ2000mm,DEJ2000ss,Flux,MajAxis,MinAxis,PA,Res,P_Flux,Field
0,00,00,0.09,-34,07,9.5,2.7,<59.8,<52.4,0.77,-61.1,C0000M36,511.92
1,00,00,0.13,-38,26,28.5,3.1,<117.,<51.7,-0.38,C0000M40,511.90,887.05
2,00,00,0.16,-04,02,42.3,106.8,<18.5,<16.4,0.34,9.9,C0000M04,511.84
3,00,00,0.18,+32,12,33.3,3.7,<43.8,<41.0,0.62,-16.1,C0000P32,511.84
4,00,00,0.19,+49,18,12.6,2.4,<93.8,<58.4,0.53,47.9,C0000P48,511.87


In [180]:
df = df.drop(columns=['Res', 'P_Flux', 'Field'])

In [181]:
df['MajAxis'][df['MajAxis'].str.contains('<')] = df['MajAxis'][df['MajAxis'].str.contains('<')].str[1:]

In [182]:
df['MinAxis'][df['MinAxis'].str.contains('<')] = df['MinAxis'][df['MinAxis'].str.contains('<')].str[1:]

In [189]:
df['dec_sign'] = pd.to_numeric(df['DEJ2000dd'].str[0]+'1')

In [190]:
df = df.apply(pd.to_numeric, errors='coerce')

In [191]:
df = df.dropna()

In [192]:
df.describe()

,RAJ2000hh,RAJ2000mm,RAJ2000ss,DEJ2000dd,DEJ2000mm,DEJ2000ss,Flux,MajAxis,MinAxis,PA,dec_sign
count,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06,1.691533e+06
mean,1.144778e+01,2.946109e+01,2.997461e+01,1.242243e+01,2.938493e+01,2.999516e+01,2.094527e+01,5.399437e+01,3.753791e+01,4.727864e-01,2.278661e-01
std,6.910262e+00,1.730936e+01,1.732430e+01,3.064593e+01,1.732670e+01,1.732805e+01,9.669644e+01,3.174119e+01,1.681311e+01,2.094749e+01,9.736928e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,-4.000000e+01,0.000000e+00,0.000000e+00,2.000000e+00,1.390000e+01,1.390000e+01,-9.000000e+01,-1.000000e+00
25%,5.000000e+00,1.400000e+01,1.498000e+01,-1.200000e+01,1.400000e+01,1.500000e+01,3.300000e+00,2.920000e+01,2.280000e+01,-1.700000e-01,-1.000000e+00
50%,1.100000e+01,2.900000e+01,2.997000e+01,1.000000e+01,2.900000e+01,3.000000e+01,5.800000e+00,4.740000e+01,3.570000e+01,2.000000e-01,1.000000e+00
75%,1.700000e+01,4.400000e+01,4.498000e+01,3.600000e+01,4.400000e+01,4.500000e+01,1.440000e+01,6.930000e+01,5.030000e+01,6.600000e-01,1.000000e+00
max,2.300000e+01,5.900000e+01,6.000000e+01,8.900000e+01,5.900000e+01,6.000000e+01,9.897100e+03,2.860000e+02,2.310000e+02,6.189800e+02,1.000000e+00


In [193]:
df = df[np.abs(df['PA'])>4]

In [194]:
df['RA'] = (df['RAJ2000hh'] + df['RAJ2000mm']/60 + df['RAJ2000ss']/3600)*15

In [197]:
df['DEC'] = df['dec_sign']*(np.abs(df['DEJ2000dd']) + df['DEJ2000mm']/60 + df['DEJ2000ss']/3600)

In [199]:
df = df.drop(columns=['RAJ2000hh', 'RAJ2000mm', 'RAJ2000ss', 
                      'DEJ2000dd', 'DEJ2000mm', 'DEJ2000ss', 'dec_sign'])

In [204]:
df = df[df['DEC']<60]

In [205]:
df.to_csv('utils/catalogues/NVSS_Catalogue.csv', index=False)

In [225]:
!ls -lah utils/catalogues/

total 396M
drwxr-xr-x 2 2020 1000  197 Nov 13 09:04 .
drwxr-xr-x 7 2020 1000  219 Nov 12 14:25 ..
-rw-r--r-- 1 2020 1000 271M Nov 12 14:24 NVSSCatalog.text
-rw-r--r-- 1 2020 1000  67M Nov 12 14:24 NVSSCatalog.text.gz
-rw-r--r-- 1 root root  14M Nov 13 09:04 NVSS_Catalogue.csv
-rw-r--r-- 1 root root  11M Nov 13 09:20 SUMSS_Catalogue.csv
-rw-r--r-- 1 2020 1000  28M Nov 12 14:23 sumsscat.Mar-11-2008
-rw-r--r-- 1 2020 1000 7.0M Nov 12 14:23 sumsscat.Mar-11-2008.gz


## Joining SUMSS and NVSS

In [257]:
sumss = pd.read_csv('utils/catalogues/SUMSS_Catalogue.csv')

In [258]:
sumss.describe()

,MajAxis,MinAxis,PA,RA,DEC,Flux
count,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000,211063.000000
mean,65.033987,50.078154,90.762212,175.803287,-53.245424,62.705213
std,17.924319,7.170397,54.965468,114.550147,15.224155,879.781546
min,0.000000,0.000000,0.000000,0.000000,-89.881889,6.000000
25%,53.600000,45.500000,41.700000,70.072042,-65.316611,12.800000
50%,60.800000,47.600000,90.800000,169.948958,-51.766333,21.200000
75%,76.500000,52.400000,139.900000,289.542646,-39.956694,45.800000
max,1156.600000,839.000000,180.000000,359.998208,-25.998528,342000.000000


In [259]:
nvss = pd.read_csv('utils/catalogues/NVSS_Catalogue.csv')

In [260]:
nvss.describe()

,Flux,MajAxis,MinAxis,PA,RA,DEC
count,256966.000000,256966.000000,256966.000000,256966.000000,256966.000000,256966.000000
mean,65.490451,44.579858,28.562285,1.655972,183.194869,7.696362
std,209.540387,29.176830,18.176351,51.475610,103.494239,27.347673
min,2.100000,13.900000,13.900000,-90.000000,0.001042,-40.430806
25%,12.200000,23.200000,18.200000,-42.100000,92.575938,-15.054257
50%,22.800000,35.000000,22.400000,6.100000,186.196479,7.261778
75%,52.100000,55.700000,31.500000,45.100000,273.604625,30.001417
max,9732.800000,286.000000,216.000000,618.980000,359.997625,59.999083


In [261]:
catalogue = pd.concat([nvss, sumss])

In [301]:
catalogue['MajAxis'] = catalogue['MajAxis']/3600
catalogue['MinAxis'] = catalogue['MinAxis']/3600

In [319]:
catalogue['Flux'] = catalogue['Flux']/1e3

In [320]:
catalogue.describe()

,DEC,Flux,MajAxis,MinAxis,PA,RA
count,468029.000000,468029.000000,468029.000000,468029.000000,468029.000000,468029.000000
mean,-19.786030,0.064234,0.014946,0.010629,41.839444,179.861551
std,37.877303,0.610867,0.007432,0.004963,69.160243,108.681563
min,-89.881889,0.002100,0.000000,0.000000,-90.000000,0.000000
25%,-49.379972,0.012500,0.009028,0.005944,-6.500000,81.469292
50%,-30.218083,0.022000,0.014528,0.012500,39.200000,179.903667
75%,11.218333,0.049300,0.019333,0.013444,87.100000,280.567000
max,59.999083,342.000000,0.321278,0.233056,618.980000,359.998208


In [321]:
catalogue.to_csv('utils/catalogues/SUMSS_NVSS_Clean.csv', index=False, float_format='%.4e')

In [322]:
!ls -lah utils/catalogues/

total 426M
drwxr-xr-x 2 2020 1000  259 Nov 13 09:58 .
drwxr-xr-x 7 2020 1000  219 Nov 12 14:25 ..
-rw-r--r-- 1 2020 1000 271M Nov 12 14:24 NVSSCatalog.text
-rw-r--r-- 1 2020 1000  67M Nov 12 14:24 NVSSCatalog.text.gz
-rw-r--r-- 1 root root  14M Nov 13 09:04 NVSS_Catalogue.csv
-rw-r--r-- 1 root root  12M Nov 13 09:34 SUMSS_Catalogue.csv
-rw-r--r-- 1 root root  30M Nov 13 12:04 SUMSS_NVSS_Clean.csv
-rw-r--r-- 1 2020 1000  260 Nov 13 10:06 get_ast_sources.py
-rw-r--r-- 1 2020 1000  28M Nov 12 14:23 sumsscat.Mar-11-2008
-rw-r--r-- 1 2020 1000 7.0M Nov 12 14:23 sumsscat.Mar-11-2008.gz


In [309]:
def inview(df, phase_centre, radius, min_flux):
    r = np.sqrt((df['RA']-phase_centre[0])**2 + (df['DEC']-phase_centre[1])**2)
    return df[(r<radius) & (df['Flux']>min_flux)]

In [310]:
df = inview(catalogue, [0, -30], 0.10, 0)

In [311]:
df.describe()

,DEC,Flux,MajAxis,MinAxis,PA,RA
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
mean,-29.955611,16.000000,0.028417,0.017389,156.250000,0.061979
std,0.050872,3.535534,0.003575,0.004439,18.172644,0.051413
min,-29.991583,13.500000,0.025889,0.014250,143.400000,0.025625
25%,-29.973597,14.750000,0.027153,0.015819,149.825000,0.043802
50%,-29.955611,16.000000,0.028417,0.017389,156.250000,0.061979
75%,-29.937625,17.250000,0.029681,0.018958,162.675000,0.080156
max,-29.919639,18.500000,0.030944,0.020528,169.100000,0.098333


In [312]:
lsm_header = '#format:name ra_d dec_d i q u v emaj_s emin_s pa_d'
lsm_array = np.empty((len(df), 10))
lsm_array[:,0] = np.arange(len(df))
lsm_array[:,1] = df['RA'].values
lsm_array[:,2] = df['DEC'].values
lsm_array[:,3] = df['Flux'].values
lsm_array[:,4:7] = 0.0
lsm_array[:,7] = df['MajAxis'].values
lsm_array[:,8] = df['MinAxis'].values
lsm_array[:,9] = df['PA'].values
np.savetxt('test_LSM.lsm', lsm_array, header=lsm_header,
           fmt='%i %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e')

In [325]:
flux = df['Flux'].values[:,None,None]

In [329]:
zeros = np.zeros((2,1,4096,4))

In [330]:
zeros[:,:,:,0] = flux

In [332]:
zeros[0,0,:,0]

array([13.5, 13.5, 13.5, ..., 13.5, 13.5, 13.5])